In [2]:
import requests
from requests.exceptions import HTTPError

robots = None
URL = 'https://www.imdb.com/robots.txt'

In [74]:
try:
    resposta = requests.get(URL)
    resposta.raise_for_status()
except HTTPError as error:
    print(error)
else:
    robots = resposta.text

In [46]:
if 'top' in robots or 'charts' in robots:
    print('True')
else:
    print('False')


False


In [391]:
conteudo = None
URL2 = 'https://www.imdb.com/chart/top/'

# Estava apresentando um Error 403... então foi necessário cria esse 'User-Agent', assim o site me permitiu acessar o contéudo...
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

try:
    resposta2 = requests.get(URL2, headers=headers)
    resposta2.raise_for_status()
    print('Página salva com sucesso!')
except HTTPError as error:
    print(error)
else:
    conteudo = resposta2.content

Página salva com sucesso!


In [392]:
from bs4 import BeautifulSoup

pagina = BeautifulSoup(conteudo, 'html.parser')

In [400]:
lista_filmes = pagina.find_all('div', attrs= {'class':'sc-b0691f29-0 jbYPfh cli-children'})
conteudo_extraido = list()

contador = 0
for filme in lista_filmes:
    dados = list()

    #Contador para parar a execução quando atingir o range de 10 filmes, se deixar sem ele o loop for vai coletar a informação de todos os 250 filmes da lista da IMDb...
    contador += 1
    if contador > 10:
        break
    
    #Ao fazer o Scraping para pegar o título o Ranking do filme vem junto em uma str, então usei o split para criar uma lista, coloquei cada valor em sua variável...
    ranking = filme.a.h3.text.split(sep='.')[0]
    titulo = filme.a.h3.text.split(sep='.')[1].strip()

    #A class abaixo corresponde ao Ano do filme, .text para pegar somente o ano.
    ano = filme.find('span', class_ = 'sc-b0691f29-8 ilsLEX cli-title-metadata-item').text

    #A class abaixo vem com a nota e tempo de duração do filme, fiz um split no final da linha de código para pegar somente a nota...
    nota = filme.find('span', class_ = 'ipc-rating-star ipc-rating-star--base ipc-rating-star--imdb ratingGroup--imdb-rating').text.split()[0]

    dados.append(ranking)
    dados.append(titulo)
    dados.append(ano)
    dados.append(nota)
    conteudo_extraido.append(dados)

print(conteudo_extraido)

[['1', 'Um Sonho de Liberdade', '1994', '9.3'], ['2', 'O Poderoso Chefão', '1972', '9.2'], ['3', 'Batman: O Cavaleiro das Trevas', '2008', '9.0'], ['4', 'O Poderoso Chefão II', '1974', '9.0'], ['5', '12 Homens e uma Sentença', '1957', '9.0'], ['6', 'A Lista de Schindler', '1993', '9.0'], ['7', 'O Senhor dos Anéis: O Retorno do Rei', '2003', '9.0'], ['8', 'Pulp Fiction: Tempo de Violência', '1994', '8.9'], ['9', 'O Senhor dos Anéis: A Sociedade do Anel', '2001', '8.9'], ['10', 'Três Homens em Conflito', '1966', '8.8']]


In [401]:
import csv

with open(file='./imdb.csv', mode='w', encoding='utf-8') as arquivo:
    arquivo_csv = csv.writer(arquivo, delimiter=';')
    arquivo_csv.writerows([['Ranking', 'Título', 'Ano', 'Nota']] + list(map(lambda lista: lista, conteudo_extraido)))
   